In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
df = pd.DataFrame(
    columns=["sick", "healthy"],
    index=["old", "young"],
    data=[[0.8, 0.2],
         [0.6, 0.4]],
)

In [3]:
df

,sick,healthy
old,0.8,0.2
young,0.6,0.4


In [10]:
def parse_exog_process_matrix(matrix, process_name):
    states, process_states = check_states_size(matrix)
    covariates = create_options_entries(states, process_name, process_states)
    transformed_matrix = transform_matrix(matrix)
    params = create_params(transformed_matrix, states, process_name, process_states)
    return params, covariates


def create_params(transformed_matrix, states, process_name, process_states):
    categories = [
        f"exogenous_process_{process_name}_{state}" for state in process_states
    ]
    index = pd.MultiIndex.from_tuples(
        itertools.product(categories, states), names=["category", "name"]
    )
    params = pd.DataFrame(index=index, columns=["value"])
    for process_state in process_states:
        params.loc[
            (f"exogenous_process_health_shock_{process_state}", states), "value"
        ] = transformed_matrix[process_state].values
    return params

    
def transform_matrix(matrix):
    transformed_matrix = matrix.copy()
    transformed_matrix[transformed_matrix == 0] = -1e300
    transformed_matrix[transformed_matrix > 0] = np.log(
        transformed_matrix[transformed_matrix > 0]
    )
    transformed_matrix.index = transformed_matrix.index.map(str)
    return transformed_matrix


def create_options_entries(states, process_name, process_states):
    covariates = {}
    for state in states:
        i = 0
        for process_state in process_states:
            if process_state in state:
                destination_state = process_state
                i += 1
        if i == 0:
            covariates[str(state)] = "?"
        elif i == 1:
            covariates[str(state)] = f"{process_name} == {destination_state} & ?"
        else:
            raise ValueError(f"{state} contains more than one process state.")
    return covariates


def check_states_size(matrix):

    return states, process_states


def check_numerics(matrix, n_states):
    assert (matrix.sum(axis=1) == np.full(n_states, 1)).all()
    assert (((matrix >= 0) & (matrix <= 1)).all()).all()

In [11]:
params, covariates  = parse_exog_process_matrix(df, "health_shock")

In [12]:
params

value
category                               name           
exogenous_process_health_shock_sick    old   -0.223144
                                       young -0.510826
exogenous_process_health_shock_healthy old    -1.60944
                                       young -0.916291

In [7]:
covariates

{'old': '?', 'young': '?'}